<a href="https://colab.research.google.com/github/samsaksham/Tranferlearningproject/blob/master/Transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:

from google.colab import drive
drive.mount('/content/drive')

from keras.applications import VGG16

img_rows = 224
img_cols = 224 

#first load the VGG16 model
basic_model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

for (i,layer) in enumerate(basic_model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [22]:


# Here we freeze the last 4 layers 
for layer in basic_model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(basic_model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)


0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [29]:

def addingTopModel(bottom_model, num_classes, D=256):
    
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return addingTop_model

In [30]:
#Lets add our FC Head back onto VGG
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 5

FC_Head = addTopModel(basic_model, num_classes)

modelnew = Model(inputs=basic_model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [17]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/My Drive/train'
validation_data_dir = '/content/drive/My Drive/val'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [18]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("image_celeb.h5", monitor="val_loss",mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3,
                          verbose = 1,restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 93
nb_validation_samples = 25
epochs = 10
batch_size = 9

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("image_celeb.h5")

Epoch 1/10
10/10 [==============================] - 96s 10s/step - loss: 11.8360 - accuracy: 0.1975 - val_loss: 2.5948 - val_accuracy: 0.3000

Epoch 00001: val_loss improved from inf to 2.59481, saving model to image_celeb.h5
Epoch 2/10
10/10 [==============================] - 84s 8s/step - loss: 2.3864 - accuracy: 0.4371 - val_loss: 4.7552 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 2.59481
Epoch 3/10
10/10 [==============================] - 87s 9s/step - loss: 2.2026 - accuracy: 0.4076 - val_loss: 4.4312 - val_accuracy: 0.0667

Epoch 00003: val_loss did not improve from 2.59481
Epoch 4/10
10/10 [==============================] - 88s 9s/step - loss: 1.6679 - accuracy: 0.4904 - val_loss: 1.6212 - val_accuracy: 0.3500

Epoch 00004: val_loss improved from 2.59481 to 1.62117, saving model to image_celeb.h5
Epoch 5/10
10/10 [==============================] - 86s 9s/step - loss: 1.0113 - accuracy: 0.6494 - val_loss: 3.0422 - val_accuracy: 0.3333

Epoch 00005: val_

In [33]:
from keras.models import load_model
classifier = load_model('image_celeb.h5')

#Testing the Model
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt

import os
import cv2
from os import listdir
from os.path import isfile, join

five_celeb_dict = {"[0]": "ben_afflek", 
                      "[1]": "elton_john",
                      "[2]": "jerry_seinfeld",
                      "[3]": "madonna",
                      "[4]": "mindy_kaling",
                     }

five_celeb_dict_n = {"ben_afflek": "ben_afflek", 
                      "elton_john": "elton_john",
                      "jerry_seinfeld": "jerry_seinfeld",
                      "madonna": "madonna",
                      "mindy_kaling": "mindy_kaling",
                      }


def draw_test(name, pred, im):
    celeb =five_celeb_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, celeb, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + five_celeb_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("/content/drive/My Drive/val/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)

    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
  
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
  

Class - jerry_seinfeld
Class - jerry_seinfeld
Class - elton_john
Class - mindy_kaling
Class - ben_afflek
Class - madonna
Class - madonna
Class - ben_afflek
Class - jerry_seinfeld
Class - mindy_kaling
